# Контекст

Есть данные о домах по округам Калифорнии. Эти данные имеют метрики, такие как население, медианный доход, средняя стоимость дома и т.д., для каждой группы блоков (округов) в Калифорнии.


# Задача

Построение модели цен на жилье в Калифорнии,модель должна обучаться на таких данных и быть способной прогнозировать среднюю 
стоимость дома в любом округе, учитывая все остальные метрики. На выходе модели должна быть предсказанная цена дома.

# Описание данных

Признаки

- longitude - географическая долгота округа          
- latitude - географическая широта округа
- housing_median_age - средний возраст дома
- total_rooms - кол-во комнат в округе   
- total_bedrooms - кол-во спален в округе   
- population - население округа  
- households - кол-во домов в округе
- median_income - медианный доход в округе (масштабированно до интервала 0.5-15)
- ocean_proximity - близость к океану

Таргет

median_house_value - стоимость дома в округе


In [1]:
# база
import numpy as np
import pandas as pd

# визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# для трансформатора
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion 

# нейросетевое конструирование
from keras import models, layers

# для итоговой проверки метрики
from sklearn.metrics import mean_absolute_error

# time
import time


RANDOM_STATE = 42

# Импорт данных / просмотр

In [2]:
data = pd.read_csv('arenda_california.txt')
data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'arenda_california.txt'

In [ ]:
data.isnull().any()

In [ ]:
data.head()

In [ ]:
# Рассмотрим расположение домов с помощью широты и долготы --> получим карту Калифорнии
plt.figure(figsize = (15,5))
sns.scatterplot(data = data,
                x = 'longitude',
                y = 'latitude',
                hue = 'ocean_proximity',
                alpha = 0.7)

In [ ]:
# расположение соответствует действительности, далее рассмотрим распределения числовых признаков
data.iloc[:,2:].hist(bins = 50,
          figsize = (15,10))

In [ ]:
# Распределение признака ocean_proximity
plt.figure(figsize = (10,3))
sns.countplot(data = data,
              x = 'ocean_proximity')
plt.grid()


data['ocean_proximity'].value_counts()

In [ ]:
# посмотрим на boxplot графики (без широты/долготы/близости_к_океану)
for column_name in data.iloc[:,2:-1].columns:
    plt.figure(figsize = (10,1))
    sns.boxplot(data = data.iloc[:,2:-1],
                x = column_name)
    plt.show()

In [ ]:
# посмотрим на статистические показатели (без широты/долготы)
data.iloc[:,2:].describe().round(1)

In [ ]:
data.duplicated().sum()

## Вывод по данным

- признаки широты и долготы соответствуют дейсвительности

---

- housing_median_age:
    - признак явно ограничен числом 52 (это видно по распределению)
    - из-за этого, скорее всего в этом признаке нет выбросов
    - будем делать из этого признака категориальный, т.к. будущая модель может воспримать это как числовую границу --> если попадется дом старше 52, вероятность плохого таргетирования вырастет

---

- median_income:
    - распределение, в целом, хорошее
    - есть выбросы (в целом, приемлимые)

---

- median_house_value:
    - признак ограничен числом 500 000
    - это наш таргет, поэтому надо либо уточнять такие данные, либо их удалять --> удалять

---

Остальнае признаки имеют распределение с плавным правым хвостом, что нормально, потом сделаем распределение, близкое к нормальному

- есть один object признак - 
- есть пустые значения в признаке total_berooms
- дубликатов нет

# Подготовка данных проектирование признаков / конструирование признаков и создание трансформатора данных для гибкости смены преобразования данных по необходимости

### Разделение на тренировочный / валидационный / проверочный наборы

In [ ]:
# пока ограничимся обычным разбиение, позже можем обратить внимание на стратификацию по определенномк признаку
y = data['median_house_value']
X = data.drop('median_house_value', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = RANDOM_STATE)


X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size = 0.2,
                                                  random_state = RANDOM_STATE,
                                                  )


X_train

### housing_median_age

In [ ]:
# Воспользуемся разделение признака с помощью квантиля --> превращаем признак в категориальный
# 1 - новый 2 -средний 3 - старый
print(np.quantile(X_train['housing_median_age'], q = [0.25, 0.5, 0.75]))

def housing_median_age_transformer(x): # записываем эту функция для трансформера
        if x <= 18:
            return 1
        if x > 18 and x <= 29:
            return 2
        if x > 29 and x <= 37:
            return 3
        else:
            return 4

X_train['housing_median_age'] = X_train['housing_median_age'].map(housing_median_age_transformer)
X_train.head()

### median_income

In [ ]:
def vibros_identification_create_filter_frame(df,                           
                                              spisok_priznakov):
    
    '''
    df - датафрейм который мы фильтруем по выбросам
    spisok_main_priznakov - список названий колонок про которым мы фильтруем выбросы

    Например: 

    spisok_main_priznakov = ['цена на момент снятия с публикации (млн. руб.)',
                             'площадь квартиры в квадратных метрах (м²)',
                             'всего этажей в доме',
                             'высота потолков (м)',
                             'площадь кухни (м²)',
                             'жилая площадь (м²)']
    '''
    
    data_new = df.copy()

    for column_name in spisok_priznakov:
        q1 = df[column_name].quantile(0.25)
        q3 = df[column_name].quantile(0.75)
        IQR = q3-q1     # интерквартильный размах 


        filter_for_kolonka = (df[column_name] >= (q1 - 1.5*IQR)) & (df[column_name] <= (q3 + 1.5*IQR))   # Получаем маску признаков без выбросов

        indexes = data_new[filter_for_kolonka].index
        data_new = data_new.loc[indexes] 

        
    return data_new 


data1 = vibros_identification_create_filter_frame(X_train,
                                                  ['median_income'])

print('Длина датасета после обработки выбросов: ', len(data1))
print('Длина датасета после обработки выбросов: ', len(data))
print('Потеряли данных: ', len(X_train) - len(data1)) # Довольно большие потери данных, 
                                                   # можем оставить признак неизменным
                                                   # (позже можем изменить в трансформере)

### total_bedrooms

In [ ]:
X_train['total_bedrooms'] = SimpleImputer(strategy = 'median').fit_transform(np.array(X_train['total_bedrooms']).reshape(-1,1))
X_train.info()

### ocean_proximity

In [ ]:
X_train.head()

In [ ]:
encoder = OneHotEncoder()   # можно сразу применять для текста
house_sea_1hot_encoded =\
encoder.fit_transform(np.array(X_train['ocean_proximity']).reshape(-1,1))


one_hot_data =\
pd.DataFrame(house_sea_1hot_encoded.toarray(),
             columns = encoder.categories_)

'''X_train = pd.concat([X_train, one_hot_data],axis = 1).drop('ocean_proximity', axis = 1)
X_train.head()'''

one_hot_data

### Конструирование новых признаков

In [ ]:
X_train['rooms_per_households'] = X_train['total_rooms'] / X_train['households']
X_train['bedrooms_per_rooms'] = X_train['total_bedrooms'] / X_train['total_rooms']
X_train['population_per_households'] = X_train['population'] / X_train['households']

### Построим матрицу корреляции и узнаем признаки, которые взаимосвязанны с таргетом

In [ ]:
pd.concat([X_train, y_train], axis = 1).corr().round(2)['median_house_value'].sort_values(ascending = False)
# Довавим признак bedrooms_per_rooms, т.к. он даже лучше связан с нащим таргетом, чем total_rooms

In [ ]:
X_train = X_train.drop(['rooms_per_households', 'population_per_households'], axis = 1)
X_train.info()

# Пишем трансформер данных

In [ ]:
start_time = time.time()

# преобразование признака 'housing_median_age'
def housing_median_age_transformer(x):
    if x <= 18:
        return 1
    if x > 18 and x <= 29:
        return 2
    if x > 29 and x <= 37:
        return 3
    if x > 37:
        return 4
    
################################# Трансформатор входных значений ##########################################################
housing_age_ix, rooms_ix, bedrooms_ix = 2, 3, 4

class NumpyFeaturesAdderTransformer(BaseEstimator, TransformerMixin):
    def fit(self, features):
        return self
    def transform(self, features):
        # Перевод признака midian_age в категориальный
        housing_median_age_cat = [housing_median_age_transformer(i) for i in features[:,2]]
        features[:,2] = housing_median_age_cat
        bedrooms_per_rooms = features[:,4] / features[:,3]
        return np.c_[features, bedrooms_per_rooms]
    



class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values




num_attribs = [i for i in X.columns if X[i].dtype != 'object']
cat_attribs = ['ocean_proximity']

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy = 'median')),
    ('attrib_changer_adder', NumpyFeaturesAdderTransformer()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('one_hot_encoder', OneHotEncoder()),
])

full_pipeline = FeatureUnion(transformer_list = [
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])


X_train = full_pipeline.fit_transform(X_train)


end_time = time.time()
print('Время трансформации: ', end_time - start_time)

X_train = X_train.toarray()

In [ ]:
X_train.shape # 8 изначальных признаков (без таргета и без категориального), 1 сконструированный (bedrooms_per_rooms),
              # 5 OneHot признаков от категориального ocean_proximity

# Делаем трансформированные значения

In [ ]:
X_val = full_pipeline.fit_transform(X_val).toarray()
X_test = full_pipeline.fit_transform(X_test).toarray()

print('Размерность валидационного набора: ', X_val.shape)
print('Размерность проверочного набора: ', X_test.shape)

# Модель

In [ ]:
import tensorflow as tf
import keras


def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(((y_pred, y_true)**2)))

def builder_model():
    model = models.Sequential()
    model.add(layers.Dense(16, activation = 'relu', input_shape = (X_train.shape[1],)))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'rmsprop',
                  loss = 'mse',
                  metrics = 'mae')
    return model


# Пробуем первую модель

In [ ]:
model = builder_model()
history =\
model.fit(X_train,
          y_train,
          batch_size = 20,
          epochs = 500,
          verbose = False,
          validation_data = (X_val, y_val))

In [ ]:
data_history =\
pd.DataFrame(history.history)

# LOSS
sns.scatterplot(data_history[-50:],
                x = data_history[-50:].index,
                y = 'loss',
                label = 'mse потери на этапе обучения')
'''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба 
'''
sns.lineplot(data = data_history[-50:],
             x = data_history[-50:].index,
             y = 'val_loss',
             color = 'green',
             label = 'mse потери на этапе проверки')

plt.legend()
plt.xlabel('эпоха')
plt.title('Потери на тренировке и тестировании')
plt.grid()
plt.show()




# ACCURACY
sns.scatterplot(data_history[-50:],
                x = data_history[-50:].index,
                y = 'mae',
                label = 'MAE на этапе обучения')
'''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба '''

sns.lineplot(data = data_history[-50:],
             x = data_history[-50:].index,
             y = 'val_mae',
             color = 'green',
             label = 'MAE на этапе проверки')
plt.legend()
plt.grid()
plt.xlabel('эпоха')
plt.title('MAE на тренировке и тестировании')
plt.show()

data_history

Вывод по первой модели: 

- 500 эпох не достаточно для наилучших показателей модели: видим плавное уменьшение как потерь так и метрики
- переобучение не наблюдается --> вводить штрафы / dropout нецелесообразно
- увеличим количество эпох 

# Вторая модель

In [ ]:
def plot_loss_metric(history):
        data_history = pd.DataFrame(history.history)
        history_columns = data_history.columns
        # LOSS
        sns.scatterplot(data_history,
                        x = data_history.index,
                        y = 'loss',
                        label = 'потери на этапе обучения')
        '''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба 
        '''
        sns.lineplot(data = data_history,
                     x = data_history.index,
                     y = 'val_loss',
                     color = 'green',
                     label = 'потери на этапе проверки')

        plt.legend()
        plt.xlabel('эпоха')
        plt.title('Потери на тренировке и тестировании')
        plt.grid()
        plt.show()

        # Metric
        sns.scatterplot(data_history,
                        x = data_history.index,
                        y = history_columns[1],
                        label = f'{history_columns[1]} на этапе обучения')
        '''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба '''

        sns.lineplot(data = data_history,
                     x = data_history.index,
                     y = history_columns[-1],
                     color = 'green',
                     label = f'{history_columns[1]} на этапе проверки')
        plt.legend()
        plt.grid()
        plt.xlabel('эпоха')
        plt.title('MAE на тренировке и тестировании')
        plt.show()

In [ ]:
data_history = pd.DataFrame(history.history)
data_history

In [ ]:
model = builder_model()
history =\
model.fit(X_train, y_train,
          batch_size = 16,
          epochs = 1000,
          verbose = False,
          validation_data = (X_val, y_val))
plot_loss_metric(history)

In [ ]:
data_history = pd.DataFrame(history.history)
data_history

In [ ]:
data_to_plot = data_history[-200:]

def plot_loss_metric_1(data):
        data_history = data
        history_columns = data.columns
        # LOSS
        sns.scatterplot(data_history,
                        x = data_history.index,
                        y = 'loss',
                        label = 'потери на этапе обучения')
        '''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба 
        '''
        sns.lineplot(data = data_history,
                     x = data_history.index,
                     y = 'val_loss',
                     color = 'green',
                     label = 'потери на этапе проверки')

        plt.legend()
        plt.xlabel('эпоха')
        plt.title('Потери на тренировке и тестировании')
        plt.grid()
        plt.show()

        # Metric
        sns.scatterplot(data_history,
                        x = data_history.index,
                        y = history_columns[1],
                        label = f'{history_columns[1]} на этапе обучения')
        '''plt.xticks(ticks = np.arange(0,21,2)) # регулировка масштаба '''

        sns.lineplot(data = data_history,
                     x = data_history.index,
                     y = history_columns[-1],
                     color = 'green',
                     label = f'{history_columns[1]} на этапе проверки')
        plt.legend()
        plt.grid()
        plt.xlabel('эпоха')
        plt.title('MAE на тренировке и тестировании')
        plt.show()

plot_loss_metric_1(data_to_plot)



Вывод по второй модели:


- 1000 эпох не достаточно для наилучших показателей модели: все еще видим уже не плавное, но все же уменьшение как потерь так и метрики
- переобучение не наблюдается --> вводить штрафы / dropout нецелесообразно
- увеличим количество эпох и добавим callback на 500 эпох

In [ ]:
callback = keras.callbacks.EarlyStopping(
    monitor = 'val_mae',
    min_delta = 500,
    patience = 500,
    verbose = False
)

In [ ]:
model = builder_model()
history =\
model.fit(X_train, y_train,
          batch_size = 16,
          epochs = 3000,
          verbose = False,
          callbacks = callback,
          validation_data = (X_val, y_val)) 
plot_loss_metric(history)

In [ ]:
# Посмотрим на последние 1000 эпох
data_last_100__200 = pd.DataFrame(history.history)[-1000:]
plot_loss_metric_1(data_last_100__200)
data_history = pd.DataFrame(history.history)
data_history

# Итог по новой модели

- замечаем, что MAE на этапе проверки сшлаживается до 39000 и не уменьшается
- в тоже время метрика на обучении уменьшается - что яв. ярким признаком переобучения на тренировочном наборе
- останавливаем обучение

# Проверка на тестовом наборе

In [ ]:
y_pred = model.predict(X_test)

mean_absolute_error(y_true = y_test,
                    y_pred = y_pred)


# Итог

Оптимальная Keras модель показывает mae = 38 823 $